# Búsqueda Best-First (variante de Costo Uniforme) — Tutorial paso a paso

Este notebook presenta una implementación clara y didáctica del algoritmo de **Búsqueda Best-First** utilizando una **cola de prioridad**.

En esta configuración se usa:
- **f(n) = costo\_del\_camino(n)**

Por lo tanto, el algoritmo se comporta como una **Búsqueda de Costo Uniforme** (equivalente a Dijkstra sobre un grafo).



## 0) Importaciones
Usaremos el módulo estándar `heapq` de Python para implementar una cola de prioridad tipo min-heap.


In [ ]:
import heapq

## 1) Definición de la clase `Node`

Un nodo de búsqueda almacena:
- `state`: el estado actual (ciudad)
- `parent`: referencia al nodo padre (para reconstruir el camino)
- `action`: acción que llevó a este nodo
- `path_cost`: costo acumulado desde el estado inicial

Se define el método `__lt__` para permitir comparaciones seguras dentro del heap.


In [ ]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        # Necesario para desempates en el heap
        return self.path_cost < other.path_cost


## 2) Función de expansión: `expand(problem, node)`

La expansión genera nodos sucesores siguiendo estos pasos:
1. Obtener las acciones disponibles desde el estado actual
2. Aplicar cada acción para obtener el siguiente estado
3. Calcular el nuevo costo acumulado
4. Generar los nodos hijos


In [ ]:
def expand(problem, node):
    s = node.state
    for action in problem.actions(s):
        s_prime = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s_prime)
        yield Node(state=s_prime, parent=node, action=action, path_cost=cost)


## 3) Núcleo del algoritmo: `best_first_search(problem, f)`

Esquema del algoritmo:
1. Inicializar la frontera con el nodo inicial priorizado por `f(n)`.
2. Mantener un diccionario `reached` con el mejor nodo encontrado por estado.
3. Repetir:
   - Extraer el mejor nodo de la frontera
   - Si es el objetivo, retornar la solución
   - Si no, expandirlo e insertar sucesores mejores


In [ ]:
def best_first_search(problem, f):
    node = Node(state=problem.initial)

    frontier = [(f(node), node)]
    heapq.heapify(frontier)

    reached = {problem.initial: node}

    while frontier:
        _, node = heapq.heappop(frontier)

        if problem.is_goal(node.state):
            return node

        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                heapq.heappush(frontier, (f(child), child))

    return None


## 4) Definición de la clase `Problem`

Esta clase encapsula los elementos específicos del dominio del problema:
- acciones posibles
- función de transición
- función de costo
- prueba de objetivo


In [ ]:
class Problem:
    def __init__(self, initial, goal, actions, result, action_cost, is_goal):
        self.initial = initial
        self.goal = goal
        self.actions = actions
        self.result = result
        self.action_cost = action_cost
        self.is_goal = is_goal


## 5) Ejemplo: rutas en el mapa de Rumania

Modelamos el problema como un grafo donde:
- `ACTIONS[ciudad]` devuelve las ciudades vecinas
- `COSTS[(ciudad, vecina)]` representa la distancia

La función `result` devuelve directamente la ciudad destino.


In [ ]:
ACTIONS = {
    "Arad": ["Sibiu", "Timisoara", "Zerind"],
    "Zerind": ["Arad", "Oradea"],
    "Oradea": ["Zerind", "Sibiu"],
    "Sibiu": ["Arad", "Oradea", "Fagaras", "Rimnicu Vilcea"],
    "Timisoara": ["Arad", "Lugoj"],
    "Lugoj": ["Timisoara", "Mehadia"],
    "Mehadia": ["Lugoj", "Drobeta"],
    "Drobeta": ["Mehadia", "Craiova"],
    "Craiova": ["Drobeta", "Rimnicu Vilcea", "Pitesti"],
    "Rimnicu Vilcea": ["Sibiu", "Craiova", "Pitesti"],
    "Fagaras": ["Sibiu", "Bucharest"],
    "Pitesti": ["Rimnicu Vilcea", "Craiova", "Bucharest"],
    "Bucharest": ["Fagaras", "Pitesti"]
}

COSTS = {
    ("Arad", "Sibiu"): 140, ("Sibiu", "Arad"): 140,
    ("Arad", "Timisoara"): 118, ("Timisoara", "Arad"): 118,
    ("Arad", "Zerind"): 75, ("Zerind", "Arad"): 75,
    ("Zerind", "Oradea"): 71, ("Oradea", "Zerind"): 71,
    ("Oradea", "Sibiu"): 151, ("Sibiu", "Oradea"): 151,
    ("Sibiu", "Fagaras"): 99, ("Fagaras", "Sibiu"): 99,
    ("Sibiu", "Rimnicu Vilcea"): 80, ("Rimnicu Vilcea", "Sibiu"): 80,
    ("Timisoara", "Lugoj"): 111, ("Lugoj", "Timisoara"): 111,
    ("Lugoj", "Mehadia"): 70, ("Mehadia", "Lugoj"): 70,
    ("Mehadia", "Drobeta"): 75, ("Drobeta", "Mehadia"): 75,
    ("Drobeta", "Craiova"): 120, ("Craiova", "Drobeta"): 120,
    ("Craiova", "Rimnicu Vilcea"): 146, ("Rimnicu Vilcea", "Craiova"): 146,
    ("Craiova", "Pitesti"): 138, ("Pitesti", "Craiova"): 138,
    ("Rimnicu Vilcea", "Pitesti"): 97, ("Pitesti", "Rimnicu Vilcea"): 97,
    ("Fagaras", "Bucharest"): 211, ("Bucharest", "Fagaras"): 211,
    ("Pitesti", "Bucharest"): 101, ("Bucharest", "Pitesti"): 101
}

def actions_fn(state):
    return ACTIONS.get(state, [])

def result_fn(state, action):
    return action

def action_cost_fn(state, action, next_state):
    return COSTS.get((state, action), float("inf"))

def is_goal_fn(state, goal="Bucharest"):
    return state == goal


## 6) Función de evaluación `f(n)`

Para obtener el comportamiento de costo uniforme:
- **f(n) = costo\_del\_camino(n)**


In [ ]:
def f_costo_uniforme(node):
    return node.path_cost


## 7) Ejecutar la búsqueda


In [ ]:
initial = "Arad"
goal = "Bucharest"

problem = Problem(
    initial=initial,
    goal=goal,
    actions=actions_fn,
    result=result_fn,
    action_cost=action_cost_fn,
    is_goal=lambda s: is_goal_fn(s, goal=goal)
)

solution = best_first_search(problem, f_costo_uniforme)
solution


## 8) Reconstrucción del camino


In [ ]:
def reconstruir_camino(goal_node):
    if goal_node is None:
        return None
    path = []
    node = goal_node
    while node is not None:
        path.append(node.state)
        node = node.parent
    path.reverse()
    return path

path = reconstruir_camino(solution)
path


## 9) Mostrar solución y costo total


In [ ]:
if solution is None:
    print("No se encontró solución")
else:
    print("Camino solución:", path)
    print("Costo total:", solution.path_cost)


## 10) Opcional: Greedy Best-First y A*

Si se define una heurística `h(n)`:
- Greedy: `f(n) = h(n)`
- A*: `f(n) = g(n) + h(n)`


In [ ]:
H = {
    "Arad": 366,
    "Zerind": 374,
    "Oradea": 380,
    "Sibiu": 253,
    "Timisoara": 329,
    "Lugoj": 244,
    "Mehadia": 241,
    "Drobeta": 242,
    "Craiova": 160,
    "Rimnicu Vilcea": 193,
    "Fagaras": 176,
    "Pitesti": 100,
    "Bucharest": 0
}

def f_greedy(node):
    return H.get(node.state, float("inf"))

def f_astar(node):
    return node.path_cost + H.get(node.state, float("inf"))

greedy_solution = best_first_search(problem, f_greedy)
astar_solution = best_first_search(problem, f_astar)

print("Greedy:", reconstruir_camino(greedy_solution))
print("A*:", reconstruir_camino(astar_solution))
